Uploading Phase:

In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install scikit-learn

Calling the Necessary Libraries:

In [2]:
import os  # For handling file paths
import cv2  # OpenCV library for reading and processing images
import numpy as np  # For numerical operations and arrays
from glob import glob  # For finding file patterns in directories
from skimage import exposure  # For image enhancement functions like histogram equalization

Finding Height and Width of an Image:

In [3]:
sample_image_path = '/content/drive/MyDrive/FACE IMAGES/1.1.pgm'

# Load the image in grayscale
img = cv2.imread('/content/drive/MyDrive/FACE IMAGES/1.1.pgm', cv2.IMREAD_GRAYSCALE)

# Print its shape
print("Image shape:", img.shape)

height = img.shape[0]
print("Height:", height)

Image shape: (80, 64)
Height: 80


Parameters:

In [4]:
IMAGE_HEIGHT = 80  # Height of each image in pixels
IMAGE_WIDTH = 64  # Width of each image in pixels
NUM_SUBJECTS = 250  # Total number of individuals in the dataset
TRAIN_PER_SUBJECT = 2  # Number of images per individual for training (e.g., 1.1, 1.2)
TEST_PER_SUBJECT = 2  # Number of images per individual for testing (e.g., 1.3, 1.4)
DATASET_PATH = '/content/drive/MyDrive/FACE IMAGES'  # Path to the folder containing all image files
IMAGE_EXTENSION = '.pgm'  # File extension of the images (.pgm or .jpg)


Loading the Training Images:

In [5]:
train_images = []  # list of 1D flattened images
train_labels = []  # list of subject IDs

for subject_id in range(1, NUM_SUBJECTS + 1):
    for sample_id in range(1, TRAIN_PER_SUBJECT + 1):
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"
        img_path = os.path.join(DATASET_PATH, filename)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Could not load image {filename}")
            continue

        # Histogram equalization
        img_eq = exposure.equalize_hist(img).astype(np.float64)

        # Mean-variance normalization
        img_norm = (img_eq - np.mean(img_eq)) / np.std(img_eq)

        # Flatten and append
        train_images.append(img_norm.flatten())
        train_labels.append(subject_id)

# Convert to numpy arrays after collecting data
train_images = np.array(train_images)  # shape: (N, pixels)
train_labels = np.array(train_labels)  # shape: (N,)


PCA Training:

In [6]:
# Calculate the average face (mean vector of all training images)
avg_face = np.mean(train_images, axis=0)

# Center each training image by subtracting the average face
centered_images = train_images - avg_face

# Compute the covariance matrix of the centered data
cov_matrix = np.cov(centered_images, rowvar=False)

# Compute eigenvalues and eigenvectors of the covariance matrix
eig_vals, eig_vecs = np.linalg.eigh(cov_matrix)

# Sort eigenvectors in descending order based on their corresponding eigenvalues
sorted_indices = np.argsort(-eig_vals)
eig_vecs = eig_vecs[:, sorted_indices]  # Rearrange eigenvectors

# Project all training images into the new eigenspace
eigenspace_train = centered_images @ eig_vecs  # Shape: (500, num_components)


 PCA Testing:

In [7]:
correct_matches = 0  # Counter for correctly recognized test images
total_tests = NUM_SUBJECTS * TEST_PER_SUBJECT  # Total number of test images = 250 × 2 = 500

# Loop through each subject
for subject_id in range(1, NUM_SUBJECTS + 1):
    # For each subject, use their third and fourth images for testing
    for sample_id in range(TRAIN_PER_SUBJECT + 1, TRAIN_PER_SUBJECT + TEST_PER_SUBJECT + 1):  # e.g., 1.3 and 1.4
        filename = f"{subject_id}.{sample_id}{IMAGE_EXTENSION}"  # Construct test filename
        img_path = os.path.join(DATASET_PATH, filename)  # Full path to image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load grayscale image

        if img is None:
            print(f"Could not load test image {filename}")  # Warn if missing
            continue

        # Preprocess test image: equalization and normalization
        img_eq = exposure.equalize_hist(img).astype(np.float64)
        img_norm = (img_eq - np.mean(img_eq)) / np.std(img_eq)
        test_vector = img_norm.flatten()  # Flatten to vector
        centered_test = test_vector - avg_face  # Subtract average face

        # Project the test image into the eigenspace
        proj_test = centered_test @ eig_vecs  # Shape: (1 x num_components)

        # -------------------------
        # RECOGNITION USING MANHATTAN DISTANCE
        # -------------------------

        # Calculate Manhattan distances between test image and all training images
        distances = np.sum(np.abs(eigenspace_train - proj_test), axis=1)

        # Find the index of the training image with the minimum distance
        closest_index = np.argmin(distances)

        # Get the predicted subject ID using the closest match
        predicted_id = train_labels[closest_index]

        # Print result
        print(f"Test image: {filename} → Predicted: {predicted_id}, Actual: {subject_id}")

        # Check if prediction is correct
        if predicted_id == subject_id:
            correct_matches += 1

Test image: 1.3.pgm → Predicted: 1, Actual: 1
Test image: 1.4.pgm → Predicted: 1, Actual: 1
Test image: 2.3.pgm → Predicted: 2, Actual: 2
Test image: 2.4.pgm → Predicted: 140, Actual: 2
Test image: 3.3.pgm → Predicted: 3, Actual: 3
Test image: 3.4.pgm → Predicted: 3, Actual: 3
Test image: 4.3.pgm → Predicted: 4, Actual: 4
Test image: 4.4.pgm → Predicted: 4, Actual: 4
Test image: 5.3.pgm → Predicted: 5, Actual: 5
Test image: 5.4.pgm → Predicted: 5, Actual: 5
Test image: 6.3.pgm → Predicted: 6, Actual: 6
Test image: 6.4.pgm → Predicted: 6, Actual: 6
Test image: 7.3.pgm → Predicted: 7, Actual: 7
Test image: 7.4.pgm → Predicted: 7, Actual: 7
Test image: 8.3.pgm → Predicted: 8, Actual: 8
Test image: 8.4.pgm → Predicted: 8, Actual: 8
Test image: 9.3.pgm → Predicted: 9, Actual: 9
Test image: 9.4.pgm → Predicted: 9, Actual: 9
Test image: 10.3.pgm → Predicted: 10, Actual: 10
Test image: 10.4.pgm → Predicted: 10, Actual: 10
Test image: 11.3.pgm → Predicted: 220, Actual: 11
Test image: 11.4.pgm →

Accuracy Report

In [8]:
accuracy = (correct_matches / total_tests) * 100
print(f"\nRecognition Accuracy: {accuracy:.2f}% ({correct_matches}/{total_tests})")


Recognition Accuracy: 92.60% (463/500)
